# Question 5

## Q5.A
**States:** $𝑠$ $∈$ $𝒮$, where $𝑆_{𝑡}$ $∈$ $𝒮$ is a realized state at time epoch $𝑡$ is represented as: $(I_{t}^{(1)}, I_{t}^{(2)}, I_{t}^{(3)})$ where $I_{t}^{i}$ is the inventory level of store $i$ at time $t$. <br>

**Actions:** $𝑎$ $∈$ $𝒜(𝑆_{𝑡})$ , where $𝐴_{𝑡}$ $∈$ $𝒜(𝑆_{𝑡})$ is a realized action at time $𝑡$. In our case, the actions are represented as a tuple of $(a_{t}^{(1)}, a_{t}^{(2)}, a_{t}^{(3)})$ where $a_{t}^{i}$ is the replenishment ordered from store $i$ at time $t$.<br>

**Demands:** Demands are distributed according to a gamma distribution with: 

Index |Mean | Variance
--- |--- | ---
**1** | 3 | 1
**2** | 5 | 2 
**3** | 2 | 3

<br>
Therefore, resulting state after taking an action is stochastic for this problem since the resulting inventory depends on the demand occurred at time $t$. It can be formally expressed as:

$𝑆_{𝑡+1} = s + a - d $ where \\
$s = (I_{t}^{(1)}, I_{t}^{(2)}, I_{t}^{(3)})$ \\
$a = (a_{t}^{(1)}, a_{t}^{(2)}, a_{t}^{(3)})$ \\
$d = (d_{t}^{(1)}, d_{t}^{(2)}, d_{t}^{(3)})$ \\

**Rewards:** There are 3 types of costs involved in the system: <br>
1. Route cost 

**Route** |(1)|(2)|(3)|(1,2)|(1,3)|(2,3)|(1,2,3)
--- |--- |--- |--- |--- |--- |--- |--- 
**Costs** |40|40|55|60|75|75|500

2. Loss sales penalty (19€ per pallet)
3. Holding cost (1€ per pallet and per day)

While the route cost is only associated with the action taken, loss sales penalty and holding cost depends both on the action and the demand. The reward function, therefore; can be formally written as:

$R_{𝑡} = -routecost_{t} + \sum_{i=1}^{3} max(0, I_{t}^{(i)}+a_{t}^{(i)}-d_{t}^{(i)})\times(-1)+min(0, I_{t}^{(i)}+a_{t}^{(i)}-d_{t}^{(i)})\times19 $

where $routecost_{t}$ represents the cost regarding the route used at time $t$ and $I_{t}^{(i)}+a_{t}^{(i)}-d_{t}^{(i)}$ represents the resulting inventory of store i at time t after the 

action is taken and the demand is deducted.


## Q5.B - Policy Evaluation

#### Step 1. Importing the libraries

In [1]:
import numpy as np
from scipy.stats import gamma
import pickle

np.random.seed(1)

#### Step 2. Discretizing the demand space

In the below cell, we create a dictionary that would map demands to probabilities using the cdf of the gamma distribution. the corresponding shape and scale parameters in the gamma distributions for the mean and demand is calculated and tabulated below using the formulas
$mean = αβ$ \\
$variance = αβ^{2}$ \\

Index |Mean | Variance|α | 	β
--- |--- | ---|--- | ---
**1** | 3 | 1| 9 | 1/3
**2** | 5 | 2 | 12.5 | 0.4 
**3** | 2 | 3| 4/3 | 1.5

While discretizing the gamma distribution, since the demand was supposed to be rounded up, we used such discretization:

Demand |Interval in the cdf (probability)
--- |--- 
**0** | 0
**1** | (0, 1]  
**2** | (1, 2]
**3** | (2, 3]
**4** | (3, 4]
**5** | (4, 5]
**6** | (5, 6]
**7** | (6, 7] 
**8** | (7, 8] 
**9** | (8, 9]
**10** |(9, ∞)


In [2]:
# the dictionary to map indiviual demand distributions to probabilities (likelihoods)
# demand_prob_individual[i, j] => probability for demand being j in store i
demand_prob_individual = {}
for i in range(1, 4):
    for j in range(0, 10):
        if i == 1:
            demand_prob_individual[(i, j)] = gamma.cdf(j, 9, scale=1/3) - gamma.cdf(max(0, j-1), 9, scale=1/3)
        elif i == 2:
            demand_prob_individual[(i, j)] = gamma.cdf(j, 12.5, scale=0.4) - gamma.cdf(max(0, j-1), 12.5, scale=0.4)
        else:
            demand_prob_individual[(i, j)] = gamma.cdf(j, 4/3, scale=1.5) - gamma.cdf(max(0, j-1), 4/3, scale=1.5)
demand_prob_individual[(1, 10)] = 1 - gamma.cdf(9, 9, scale=1/3)
demand_prob_individual[(2, 10)] = 1 - gamma.cdf(9, 12.5, scale=0.4)
demand_prob_individual[(3, 10)] = 1 - gamma.cdf(9, 4/3, scale=1.5)

Using the above probabilities, below is the dictionary that calculates and stores the probability of a demand distribution happening at any time: $prob(d) = prob(d_{t}^{(1)}, d_{t}^{(2)}, d_{t}^{(3)}) = prob(d_{t}^{(1)})\times prob(d_{t}^{(2)})\times prob(d_{t}^{(3)})$ \\

In [3]:
# every combinations of demand
demands = np.array([np.array([i,j,z]) for i in range(11) for j in range(11) for z in range(11)])
demand_probs = {}
for demand in demands:
    prob1 = demand_prob_individual[(1, demand[0])]
    prob2 = demand_prob_individual[(2, demand[1])]
    prob3 = demand_prob_individual[(3, demand[2])]
    demand_probs[tuple(demand)] = prob1 * prob2 * prob3

#### Step 3. Initializing the state space, state-value function and the policy

1. State Space: All combinations of three integers between 0-10, inclusive.

2. Policy: $I_{i} = μ_{i}+1.97σ_{i}$ corresponds to:

Index |Mean | Variance| Stock (rounded up)
--- |--- | ---|---
**1** | 3 | 1| 5 
**2** | 5 | 2 | 8 
**3** | 2 | 3| 6

3. State-Value function: 0 for each state

In [4]:
# state space
states = np.array([np.array([i,j,z]) for i in range(11) for j in range(11) for z in range(11)]) # inventory before the start of the day
state_index = {}
for i, s in enumerate(states):
    state_index[tuple(s)] = i

# initial policy
pi = np.array([np.array([max(0, 5-s[0]), max(0, 8-s[1]), max(0, 6-s[2])]) for s in states])

# state-value function
V = {}
for state in states:
    V[tuple(state)] = 0

#### Step 4. Helper functions

Below are 3 helper functions and a lookup dictionary that we will use throughout this notebook.

**1. inv_r(inv_level):** given the inventory level, returns the reward related to the inventory according to the formula: $\sum_{i=1}^{3} max(0, invLevel)\times(-1)+min(0, invLevel)\times19 $ \\

**2. route_r(action):** calculates and returns the cost associated with the route with basic if-else statements.

**3. generate_demand():** Generates demand from the gamma distribution. We used this function in the Q-learning part of the solution to generate random demands for the episode. 

**4. get_actions:** Maps the given state to possible actions. Uses the limitation that the stock can be at most 10 for each store. 

In [5]:
def inv_r(inv_level):
    inv_reward1 = (max(0, inv_level[0]) * -1) + (min(0, inv_level[0]) * 19)
    inv_reward2 = (max(0, inv_level[1]) * -1) + (min(0, inv_level[1]) * 19)
    inv_reward3 = (max(0, inv_level[2]) * -1) + (min(0, inv_level[2]) * 19)
    direct_reward = inv_reward1 + inv_reward2 + inv_reward3
    return direct_reward

def route_r(action):
    if action[0] > 0 and action[1] > 0 and action[2] > 0:
        route_reward = -500
    elif action[0] > 0 and action[1] > 0:
        route_reward = -60
    elif action[0] > 0 and action[2] > 0:
        route_reward = -75
    elif action[1] > 0 and action[2] > 0:
        route_reward = -75
    elif action[0] > 0:
        route_reward = -40
    elif action[1] > 0:
        route_reward = -40
    elif action[2] > 0:
        route_reward = -55
    else:
        route_reward = 0
    return route_reward

def generate_demand():
    d1 = min(10, np.ceil(np.random.gamma(shape=9, scale=1/3)))
    d2 = min(10, np.ceil(np.random.gamma(shape=12.5, scale=0.4)))
    d3 = min(10, np.ceil(np.random.gamma(shape=4/3, scale=1.5)))
    return np.array([d1, d2, d3])

get_actions = {}
for s in states:
    max_purchase = 10 - s
    actions = np.array([np.array([i,j,z]) for i in range(max_purchase[0]+1) for j in range(max_purchase[1]+1) for z in range(max_purchase[2]+1)])
    get_actions[tuple(s)] = actions

#### Step 5. Policy Evaluation

In [6]:
def policy_evaluation(pi):
    # Policy evaluation algorithm
    print("Evaluating the policy...")
    # Looping until convergence
    while True:
        # initializing the max_diff to zero
        max_diff = 0
        # looping over all of the states
        for ind, s in enumerate(states):
            # we order pi[s], so our inventory level jumps to = s + pi[s].
            action = pi[ind]
            # getting route cost for the action
            route_reward = route_r(action)
            # inventory level after the action (before the demand is deducted)
            temp_inv_level = s + action
            # initializing the value to zero
            val = 0
            # looping over all the possible demands
            for demand in demands:
                # the probability of the demand occurring
                prob = demand_probs[tuple(demand)]
                # inventory level after the demand
                inv_level = temp_inv_level - demand 
                # resulting reward from the inventory 
                inv_reward = inv_r(inv_level)
                # total reward
                reward = route_reward + inv_reward
                # setting negative inventory values to zero
                inv_level[inv_level<0] = 0
                # updating the value
                val += prob * (reward + 0.8 * V[tuple(inv_level)])

            # updating the maximum difference
            max_diff = max(max_diff, abs(val - V[tuple(s)]))
            # updating the state-value function
            V[tuple(s)] = val

        # If diff smaller than threshold delta for all states, algorithm terminates
        if max_diff < 0.001:
            break
    return V

Running the policy evaluation for the given initial policy.

In [7]:
try:
    with open("initial_policy_V.pkl", "rb") as fl:
    # loads the pre-computed state-value function for the initial policy
        V = pickle.load(fl)
except:
    # if the file is not available, runs the policy evaluation algorithm from scratch.
    # takes approximately 15 minutes
    V = policy_evaluation(pi)
    with open("initial_policy_V.pkl", "wb") as fl:
        pickle.dump(V, fl)
print(V)

{(0, 0, 0): -2552.6629193831477, (0, 0, 1): -2552.662920049725, (0, 0, 2): -2552.662920608634, (0, 0, 3): -2552.6629216079564, (0, 0, 4): -2552.662923341867, (0, 0, 5): -2552.662926165139, (0, 0, 6): -2112.662929646475, (0, 0, 7): -1993.3722279349984, (0, 0, 8): -1864.7434349436296, (0, 0, 9): -1744.9194969886385, (0, 0, 10): -1634.6092715789903, (0, 1, 0): -2552.6629296464744, (0, 1, 1): -2552.6629306689692, (0, 1, 2): -2552.662931526128, (0, 1, 3): -2552.662933058457, (0, 1, 4): -2552.662935716363, (0, 1, 5): -2552.662940041845, (0, 1, 6): -2112.6629453709074, (0, 1, 7): -1993.3722436265548, (0, 1, 8): -1864.743450580948, (0, 1, 9): -1744.9195125547174, (0, 1, 10): -1634.609287063428, (0, 2, 0): -2552.6629453709074, (0, 2, 1): -2552.6629471438355, (0, 2, 2): -2552.6629486294355, (0, 2, 3): -2552.662951284252, (0, 2, 4): -2552.662955886166, (0, 2, 5): -2552.662963366846, (0, 2, 6): -2112.662972566123, (0, 2, 7): -1993.3722707223665, (0, 2, 8): -1864.7434775530037, (0, 2, 9): -1744.919

## Q5.C - Policy Improvement

We implement a single of the policy improvement algorithm. Given a policy, we loop through all the states and actions and calculate the values of the actions. If the action that is maximizing the reward is not equal to the action that the policy gives, we update the policy.

In [8]:
def policy_improvement(V):
    # looping over all of the states
    for ind, s in enumerate(states):
      # getting the current value of the state
        val_max = V[tuple(s)]
        
        # Exploring all the actions
        possible_actions = get_actions[tuple(s)]
        for a in possible_actions:
            # initializng the value to zero
            val = 0
            # calculating the route cost
            route_reward = route_r(a)
            # inventory level after the action (before the demand is deducted)
            temp_inv_level = s + a
            # looping over all the possible demands
            for demand in demands:
                # the probability of the demand occurring
                prob = demand_probs[tuple(demand)]
                # inventory level after the demand
                inv_level = temp_inv_level - demand 
                # resulting reward from the inventory 
                inv_reward = inv_r(inv_level)
                # total reward
                reward = route_reward + inv_reward
                # setting negative inventory values to zero
                inv_level[inv_level<0] = 0
                # updating the value
                val += prob * (reward + 0.8 * V[tuple(inv_level)])

            # updating the policy if the action improves value and and the action different from current policy
            if val > val_max and not np.array_equal(pi[ind], a):
                pi[ind] = a
                val_max = val
    return pi

In [9]:
# running the policy improvement algorithm for a single step
try:
    with open("policy_after_improvement.pkl", "rb") as fl:
      # loads the pre-computed state-value function for the initial policy
      pi = pickle.load(fl)
except:
    # if the file is not available, runs the policy evaluation algorithm from scratch.
    # takes approximately 100 minutes
    pi = policy_improvement(V)
    with open("policy_after_improvement.pkl", "wb") as fl:
        pickle.dump(pi, fl)
print(pi)

[[10  0 10]
 [10  0  9]
 [10  0  8]
 ...
 [ 0  0  2]
 [ 0  0  1]
 [ 0  0  0]]


In [10]:
pi[0]

array([10,  0, 10])

Solving this problem to optimality using the policy evaluating algorithm can take a lot of time. This is because of the large state space, action space and the demand space we are dealing with. In policy improvement algorithm for example, we iterate over all of the states, for each state, we iterate over all the possible actions, and for each action, we iterate over all the possible demands. Although we used lookup tables (dictionaries) extensively in order to reduce the time complexity, the code takes a lot of time even for a single step of the policy iteration. In policy improvement step, we iterate for around 1 billion times and in each iteration, we make calculations, lookups, etc. which significantly increases the runtime. 

## Q5.D - Tabular Q-Learning Algorithm

Initializing the Q(s, a) table with random values

In [11]:
# initializing the Q(s, a) table
np.random.seed(seed=1)
q_table = {}
for s in states:
    actions = get_actions[tuple(s)]
    q_table[tuple(s)] = {}
    for a in actions:
        q_table[tuple(s)][tuple(a)] = np.random.rand()

Loading the pre-trained q values.

**Note: We have trained the below algorithm for millions of episodes where each episode was consisting of 30 days. Instead of running the below code for millions of iterations, we advise you to use the pre-trained q-table and run the below code for a few iterations.**

In [12]:
with open("q_dict.pkl", "rb") as fl:
    q_table = pickle.load(fl)

In [13]:
# discount factor, set to zero for exploiting
# if the model is not yet trained, it should be set to a value like 0.9
epsilon = 0.
alpha = 0.2
n_episodes = 1
# iterating for a given number of episodes
# we recommend around 10 million episodes for training from scratch
# it will take around 10 hours to train from scratch
for n in range(n_episodes):
    # randomly initializing the state
    s = np.random.randint(low=0, high=11, size=3)
    r_list = []
    # if you are not going to use the pre-trained q-table,
    # we advise you to change the # of iterations to at least 10 million.
    for _ in range(30):
        # since there are no terminal states, we are using periods of 30 days as episodes
        tuple_s = tuple(s)
        # getting the state in q table
        q_dict = q_table[tuple_s]
        index = state_index[tuple_s]
        # taking a random action with probability epsilon
        if np.random.rand() < epsilon:
            A = get_actions[tuple_s]
            action = A[np.random.randint(low=0, high=len(A))]
        # otherwise, taking the action with the max expected reward 
        else:
            action = max(q_dict, key=q_dict.get)
        # generating random demand
        d = generate_demand()
        # updating the state
        new_s = s + action  - d
        # calculating the reward
        reward = route_r(action) + inv_r(new_s)
        # setting negative inventory (loss sales) to 0.
        new_s[new_s<0] = 0
        # getting the q values for the update 
        new_q = max(q_table[tuple(new_s)].values())
        tuple_a = tuple(action)
        old_q = q_dict[tuple_a]
        # updating the q table
        q_table[tuple_s][tuple_a] += alpha * (reward + 0.8 * new_q - old_q)
        # adding the observed immediate reward to a list
        r_list.append(reward)
        print(f"state: {s}, action: {action}, inv: {s+action-d}, demand: {d}, reward: {reward}")
        s = np.asarray(new_s.copy()).astype(int)
# printing the mean of the rewards in an episode
print(f"MEAN IMMEDIATE REWARD: {np.array(r_list).mean()}")

state: [7 8 4], action: (0, 0, 0), inv: [4. 4. 1.], demand: [3. 4. 3.], reward: -9.0
state: [4 4 1], action: (0, 5, 9), inv: [0. 5. 9.], demand: [4. 4. 1.], reward: -89.0
state: [0 5 9], action: (10, 5, 0), inv: [6. 4. 6.], demand: [4. 6. 3.], reward: -76.0
state: [6 4 6], action: (0, 0, 0), inv: [ 3. -1.  5.], demand: [3. 5. 1.], reward: -27.0
state: [3 0 5], action: (6, 10, 0), inv: [5. 3. 2.], demand: [4. 7. 3.], reward: -70.0
state: [5 3 2], action: (0, 7, 8), inv: [2. 5. 6.], demand: [3. 5. 4.], reward: -88.0
state: [2 5 6], action: (8, 3, 0), inv: [ 6. -1.  3.], demand: [4. 9. 3.], reward: -88.0
state: [6 0 3], action: (0, 10, 7), inv: [1. 3. 9.], demand: [5. 7. 1.], reward: -88.0
state: [1 3 9], action: (9, 7, 0), inv: [7. 5. 8.], demand: [3. 5. 1.], reward: -80.0
state: [7 5 8], action: (0, 0, 0), inv: [ 5. -2.  7.], demand: [2. 7. 1.], reward: -50.0
state: [5 0 7], action: (0, 10, 0), inv: [2. 3. 6.], demand: [3. 7. 1.], reward: -51.0
state: [2 3 6], action: (7, 6, 0), inv: [7

This is how q table looks like for state (5, 5, 5):

In [14]:
# state = (5,5,5)
# printing the expected discounted sum of rewards for comparison with the base policy.
q_table[(5, 5, 5)]

{(0, 0, 0): -317.679611827183,
 (0, 0, 1): -374.5964496501923,
 (0, 0, 2): -377.8496196323968,
 (0, 0, 3): -379.3112679341133,
 (0, 0, 4): -378.5774747189478,
 (0, 0, 5): -369.6143395261866,
 (0, 1, 0): -372.7555941289923,
 (0, 1, 1): -378.6045478236035,
 (0, 1, 2): -373.6939582380481,
 (0, 1, 3): -387.6306144299097,
 (0, 1, 4): -368.65132589142655,
 (0, 1, 5): -371.01760554533445,
 (0, 2, 0): -378.8359255972771,
 (0, 2, 1): -387.7038582439962,
 (0, 2, 2): -377.4263739688205,
 (0, 2, 3): -378.3595003942873,
 (0, 2, 4): -377.8557142191442,
 (0, 2, 5): -373.49388980852547,
 (0, 3, 0): -371.98915474504213,
 (0, 3, 1): -379.74863861174646,
 (0, 3, 2): -375.0183806520426,
 (0, 3, 3): -377.1713126292215,
 (0, 3, 4): -371.4163389068245,
 (0, 3, 5): -373.1013750142065,
 (0, 4, 0): -376.08881558992607,
 (0, 4, 1): -377.3609219233719,
 (0, 4, 2): -378.4170736308581,
 (0, 4, 3): -370.29790189197615,
 (0, 4, 4): -374.4644800011023,
 (0, 4, 5): -368.88606781922897,
 (0, 5, 0): -371.65699732634664,


As can be seen from the above results, our nearly-converged q table have better expected discounted some of rewards rewards compared to the base policy from the policy evaluation part. Base policy must supply to all three of the stores each day (since demand cannot be zero). Route cost of this supply is a lot (-500) compared to the other options which explains the differences in the results.

## Q5.E

In Q5.D, you can see a sample episode, starting with a random inventory and ran for 30 days. It is clearly visible that the route option (1,2,3) was never used. This is because the high cost (-500) associated with this route. Our algorithm learned that it is better to bear the sales loss than trying to supply to all three of the stores at the same time. 

Our algorithm also learned that facing a possible sales loss is much worse than paying for excess inventory. We can see that it preferred to order more than the expected mean demands most of the time. This is also partly because of the fact that all three of the stores cannot be supplied at the same time because of the high cost associated with it. Therefore it learned that an excess inventory should almost always be available because of this reason. In a 30 day simulation of the system below you can see that the order-up-to inventories are relatively higher than the mean of the gamma distributions as a result. 

In [15]:
# discount factor, set to zero for exploiting
# if the model is not yet trained, it should be set to a value like 0.9
epsilon = 0.
alpha = 0.2
n_episodes = 1
# iterating for a given number of episodes
# we recommend around 10 million episodes for training from scratch
# it will take around 10 hours to train from scratch
for n in range(n_episodes):
    # randomly initializing the state
    s = np.random.randint(low=0, high=11, size=3)
    ORDER_UP_TO = []
    # if you are not going to use the pre-trained q-table,
    # we advise you to change the # of iterations to at least 10 million.
    for _ in range(300):
        # since there are no terminal states, we are using periods of 30 days as episodes
        tuple_s = tuple(s)
        # getting the state in q table
        q_dict = q_table[tuple_s]
        index = state_index[tuple_s]
        # taking a random action with probability epsilon
        if np.random.rand() < epsilon:
            A = get_actions[tuple_s]
            action = A[np.random.randint(low=0, high=len(A))]
        # otherwise, taking the action with the max expected reward 
        else:
            action = max(q_dict, key=q_dict.get)
        # generating random demand
        d = generate_demand()
        # updating the state
        new_s = s + action  - d
        # calculating the reward
        reward = route_r(action) + inv_r(new_s)
        # setting negative inventory (loss sales) to 0.
        new_s[new_s<0] = 0
        # getting the q values for the update 
        new_q = max(q_table[tuple(new_s)].values())
        tuple_a = tuple(action)
        old_q = q_dict[tuple_a]
        # updating the q table
        q_table[tuple_s][tuple_a] += alpha * (reward + 0.8 * new_q - old_q)
        # adding the observed immediate reward to a list
        ORDER_UP_TO.append(s+action)
        s = np.asarray(new_s.copy()).astype(int)
# printing the mean of the rewards in an episode
print(f"ORDER-UP-TO LEVEL: {np.round(np.array(ORDER_UP_TO).mean(axis=0),2)}")

ORDER-UP-TO LEVEL: [7.26 8.47 7.04]


As can be seen, order-up-to levels are: **[7.26  8.47 7.04]**, which is higher than the means of the gamma distribution for the demand: **[3, 5, 2]**